In [2]:
import torch
import pandas as pd

from sklearn.metrics import classification_report, confusion_matrix
from deepGene_BC.model import DeepFMLite

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_df = pd.read_csv("../data/test.csv.gz",index_col=0)
X_test = test_df.drop(columns=['subtype_PAM50']).values
y_test = test_df['subtype_PAM50'].values

X_test = torch.FloatTensor(X_test).to(device=device)
y_test = torch.LongTensor(y_test).to(device=device)

In [ ]:
trained_model = DeepFMLite(
    n_features=X_test.shape[1],
    n_classes=4
)
checkpoint = torch.load("../model/trained_model/model.pth", map_location=device)
trained_model.load_state_dict(checkpoint["model_state"])
trained_model.to(device)
trained_model.eval();

In [5]:
trained_model.eval()
with torch.no_grad():
    predict_prob = torch.softmax(trained_model(X_test), dim=1)
    predict = torch.argmax(predict_prob, dim=1)
    print(classification_report(y_test.cpu().numpy(), predict.cpu().numpy()))


              precision    recall  f1-score   support

           0       0.80      0.80      0.80        49
           1       0.76      0.70      0.73        23
           2       0.86      0.80      0.83       144
           3       0.59      0.72      0.65        57

    accuracy                           0.77       273
   macro avg       0.75      0.75      0.75       273
weighted avg       0.78      0.77      0.78       273



In [6]:
trained_model.eval()
with torch.no_grad():
    predict_prob = torch.softmax(trained_model(X_test), dim=1)
    predict = torch.argmax(predict_prob, dim=1)
    cm = confusion_matrix(y_test.cpu().numpy(), predict.cpu().numpy())
    print(cm)

[[ 39   1   5   4]
 [  6  16   1   0]
 [  3   2 115  24]
 [  1   2  13  41]]


In [ ]:
predict_label = torch.argmax(predict_prob, dim=1)

probabilities = predict_prob.cpu().numpy()
true_labels = y_test.cpu().numpy()
predicted_labels = predict_label.cpu().numpy()

n_classes = probabilities.shape[1]

data = {
    'true_label': true_labels,
    'predict_label': predicted_labels
}

for i in range(n_classes):
    data[f'class{i}_prob'] = probabilities[:, i]

df = pd.DataFrame(data)

In [ ]:
df.to_csv("../results/test_results.csv")